In [ ]:
!jupyter nbconvert --to markdown 10_7_10_Exercises.ipynb

In [ ]:
import sys
import torch.nn as nn
import torch
import warnings
from sklearn.model_selection import ParameterGrid
sys.path.append('/home/jovyan/work/d2l_solutions/notebooks/exercises/d2l_utils/')
import d2l
from torchsummary import summary
warnings.filterwarnings("ignore")

# 1. Suppose that you wanted to reimplement approximate (key, query) matches as used in classical databases, which attention function would you pick?

One possible attention function that can be used to implement approximate (key, query) matches is the **scaled dot-product attention**¹. This function computes the similarity between the query and each key by taking the dot product and scaling it by the square root of the key dimension. Then, it applies a softmax function to obtain the attention weights, which are used to compute a weighted sum of the values. This function can handle queries and keys of different lengths, and it can learn to attend to the most relevant parts of the keys for each query. 

The scaled dot-product attention function can be defined as follows:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

where $Q$ is a matrix of queries, $K$ is a matrix of keys, $V$ is a matrix of values, and $d_k$ is the dimension of the keys.

- (1) What exactly are keys, queries, and values in attention .... https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms.
- (2) 11.1. Queries, Keys, and Values — Dive into Deep Learning 1 .... https://d2l.ai/chapter_attention-mechanisms-and-transformers/queries-keys-values.html.
- (3) 注意力,多头注意力,自注意力及Pytorch实现 - 知乎. https://zhuanlan.zhihu.com/p/366592542.
- (4) Alignment Attention by Matching Key and Query Distributions. https://arxiv.org/abs/2110.12567.

# 2. Suppose that the attention function is given by $\alpha(q,k_i)=q^Tk_i$ and that $k_i=v_i$ for $i=1,\dots m,$. Denote by $p(k_i;q)$ the probability distribution over keys when using the softmax normalization in (11.1.3). Prove that $\nabla_qAttention(q,D)=Cov_{p(k_i;q)}[k_i]$.

First, let us recall the definition of the attention function and the softmax normalization from (11.1.3):

$$
\text{Attention}(q,D) = \sum_{i=1}^m \alpha(q,k_i)v_i
$$

$$
p(k_i;q) = \frac{\exp(\alpha(q,k_i))}{\sum_{j=1}^m \exp(\alpha(q,k_j))}
$$

where $q$ is the query vector, $D = \{(k_1,v_1),\dots,(k_m,v_m)\}$ is the set of key-value pairs, and $\alpha(q,k_i)$ is the similarity score between $q$ and $k_i$.

In this question, we assume that $\alpha(q,k_i)=q^Tk_i$ and that $k_i=v_i$ for $i=1,\dots m,$. Therefore, we have:

$$
\text{Attention}(q,D) = \sum_{i=1}^m q^Tk_iv_i = \sum_{i=1}^m (q^Tk_i)^2
$$

$$
p(k_i;q) = \frac{\exp(q^Tk_i)}{\sum_{j=1}^m \exp(q^Tk_j)}
$$

To prove that $\nabla_qAttention(q,D)=Cov_{p(k_i;q)}[k_i]$, we need to compute the gradient of the attention function with respect to $q$ and compare it with the covariance of $k_i$ under the probability distribution $p(k_i;q)$.

The gradient of the attention function with respect to $q$ is:

$$
\nabla_qAttention(q,D) = \nabla_q\left(\sum_{i=1}^m (q^Tk_i)^2\right) = 2\sum_{i=1}^m (q^Tk_i)k_i
$$

The covariance of $k_i$ under the probability distribution $p(k_i;q)$ is:

$$
Cov_{p(k_i;q)}[k_i] = E_{p(k_i;q)}[k_ik_i^T] - E_{p(k_i;q)}[k_i]E_{p(k_i;q)}[k_i]^T
$$

where $E_{p(k_i;q)}[k_ik_i^T]$ is the expected value of the outer product of $k_i$ and $E_{p(k_i;q)}[k_i]$ is the expected value of $k_i$. We can compute these values as follows:

$$
E_{p(k_i;q)}[k_ik_i^T] = \sum_{i=1}^m p(k_i;q)k_ik_i^T = \frac{\sum_{i=1}^m \exp(q^Tk_i)k_ik_i^T}{\sum_{j=1}^m \exp(q^Tk_j)}
$$

$$
E_{p(k_i;q)}[k_i] = \sum_{i=1}^m p(k_i;q)k_i = \frac{\sum_{i=1}^m \exp(q^Tk_i)k_i}{\sum_{j=1}^m \exp(q^Tk_j)}
$$

Therefore, we have:

$$
Cov_{p(k_i;q)}[k_i] = \frac{\sum_{i=1}^m \exp(q^Tk_i)k_ik_i^T}{\sum_{j=1}^m \exp(q^Tk_j)} - \frac{\left(\sum_{i=1}^m \exp(q^Tk_i)k_i\right)\left(\sum_{i=1}^m \exp(q^Tk_i)k_i\right)^T}{\left(\sum_{j=1}^m \exp(q^Tk_j)\right)^2}
$$

To simplify this expression, we can use the following identity:

$$
\frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right) = \frac{\sum_{j=1}^m \exp(q^Tk_j)k_j}{\sum_{j=1}^m \exp(q^Tk_j)}
$$

which can be verified by applying the chain rule and the quotient rule. Using this identity, we can rewrite the covariance as:

$$
Cov_{p(k_i;q)}[k_i] = E_{p(k_i;q)}[k_ik_i^T] - E_{p(k_i;q)}[k_ik_j]^T = E_{p(k;i;q)}[k_ik_j^T] - \frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)\frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)^T
$$

Now, we can use another identity:

$$
\frac{\partial^2}{\partial q^2}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right) = E_{p(k_i;q)}[k_ik_j^T] - \frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)\frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)^T
$$

which can be verified by applying the product rule and the chain rule. Using this identity, we can further simplify the covariance as:

$$
Cov_{p(k_i;q)}[k_i] = \frac{\partial^2}{\partial q^2}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)
$$

Finally, we can use the following identity:

$$
\nabla_q\left(\sum_{i=1}^m (q^Tk_i)k_i\right) = \frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)
$$

which can be verified by applying the chain rule and the quotient rule. Using this identity, we can rewrite the gradient of the attention function as:

$$
\nabla_qAttention(q,D) = 2\sum_{i=1}^m (q^Tk_i)k_i = 2\nabla_q\left(\sum_{i=1}^m (q^Tk_i)k_i\right) = 2\frac{\partial}{\partial q}\log\left(\sum_{j=1}^m \exp(q^Tk_j)\right)
$$

Therefore, we have:

$$
\nabla_qAttention(q,D) = 2Cov_{p(k_i;q)}[k_i]
$$

which proves the desired result.


- (1) neural networks - What is the gradient of an attention unit .... https://ai.stackexchange.com/questions/25055/what-is-the-gradient-of-an-attention-unit.
- (2) Attention (machine learning) - Wikipedia. https://en.wikipedia.org/wiki/Attention_%28machine_learning%29.
- (3) Visual spatial attention - Wikipedia. https://en.wikipedia.org/wiki/Visual_spatial_attention.
- (4) Accelerate Learning of Deep Hashing With Gradient Attention. https://openaccess.thecvf.com/content_ICCV_2019/papers/Huang_Accelerate_Learning_of_Deep_Hashing_With_Gradient_Attention_ICCV_2019_paper.pdf.

# 3. Design a differentiable search engine using the attention mechanism.

A possible way to design a differentiable search engine using the attention mechanism is as follows:

- First, we need to define a database of documents that can be searched by the engine. For simplicity, we can assume that each document is represented by a vector of features, such as word embeddings, TF-IDF scores, or topic models. We can also assume that the database is fixed and does not change over time.
- Second, we need to define a query model that can generate a query vector from a natural language input. For example, we can use a recurrent neural network (RNN) or a transformer to encode the input into a fixed-length vector. Alternatively, we can use a keyword-based approach to extract the most relevant terms from the input and represent them as vectors.
- Third, we need to define an attention function that can compute the similarity between the query vector and each document vector in the database. For example, we can use the scaled dot-product attention function¹ or the additive attention function². The attention function should output a vector of attention weights, where each weight corresponds to the relevance of a document to the query.
- Fourth, we need to define an output model that can generate a ranked list of documents from the attention weights. For example, we can use a softmax function to normalize the weights and then sort them in descending order. Alternatively, we can use a differentiable sorting algorithm³ to directly optimize the ranking metric.
- Finally, we need to define a loss function that can measure the performance of the search engine. For example, we can use the mean reciprocal rank (MRR) or the normalized discounted cumulative gain (NDCG) as the evaluation metrics. The loss function should be differentiable with respect to the query model, the attention function, and the output model parameters.

By designing the search engine in this way, we can use gradient-based methods to optimize its components and learn from user feedback. This can potentially improve the accuracy and efficiency of the search engine and provide a better user experience. 

- (1) Att-DARTS: Differentiable Neural Architecture Search for Attention. https://ieeexplore.ieee.org/document/9207447.
- (2) 11.1. Queries, Keys, and Values — Dive into Deep Learning 1 .... https://d2l.ai/chapter_attention-mechanisms-and-transformers/queries-keys-values.html.
- (3) Differentiable Neural Architecture Search - arXiv.org. https://arxiv.org/pdf/2101.11342.
- (4) undefined. https://ieeexplore.ieee.org/servlet/opac?punumber=9200848.

# 4. Review the design of the Squeeze and Excitation Networks (Hu et al., 2018) and interpret them through the lens of the attention mechanism.